In [2]:
import nltk
nltk.download('stopwords')
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jfuentes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from ourDb import events_collection, total_events_collection, events_ml_collection
import re
import nltk
import string
from tqdm import tqdm
import numpy as np
from nltk.tokenize.moses import MosesTokenizer
from collections import Counter


from gensim.models import Phrases
from nltk.corpus import wordnet as wn

In [4]:
class PreprocessText:
    def __init__(self, categorizedEvents):
        """categorizedEvents should be a list of dictionaries each corresponding to an event
            X is the tokenized preprocessed text
            Y is the corresponding categories
            phraseMl is the phrase model that can further trained and used
            phrases is a list of all the phrases identified"""
        self.X = []
        self.Y = []
        tokenizer = MosesTokenizer() #tokenizers are basically an advanced split
        for e in categorizedEvents:
            text = e[u'name'] + " " + e[u'description']
            text = tokenizer.tokenize(text)
            text = self.preprocess(text)
            self.X.append(text)
            self.Y.append(e[u'category'])
        self.phraseMl = Phrases(self.X, min_count=3) #learn ml model for phrase
        self.X = list(self.phraseMl[self.X]) #use ml model for phrases
#         self.X = list(self.phraseMl[self.X]) #get triples
        self.phrases = phrases = set([w for doc in self.X for w in doc if '_' in w])
        
    def matchNotX(self, strg, search=re.compile(r'[^!#$%&()*+,-./:;<=>?@\\^_}|{~0123456789]').search):
        """make sure word has something than punctuation"""
        return bool(search(strg)) #make sure word has something other than punctuation

    def preprocess(self, text):
        """Remove all useless words and punct, make lowercase"""
        stoplist = set('for a of the and to in . / '.split())
        stoplist = set(nltk.corpus.stopwords.words('english')) | stoplist | set(string.punctuation)
        return [word.strip(string.punctuation).lower() for word in text if word not in stoplist and self.matchNotX(word)]    
        
    def topBigrams(self, texts, n, tri=False):
        """Other method of getting phrases, currently unused because phrases can be further trained(online) and saved"""
        flatTexts = []
        for text in texts:
            for word in text:
                flatTexts.append(word)
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        trigram_measures = nltk.collocations.TrigramAssocMeasures()
        topAnswers = []
        if tri:
            finder = nltk.collocations.TrigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(trigram_measures.pmi, n)
        else:
            finder = nltk.collocations.BigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(bigram_measures.pmi, n)

In [113]:
def gatherCategorizedEvents():
    allCategorizedEvents = []
    allEvents = events_ml_collection.find({}, {"category": 1, "description": 1, "name": 1, "_id": 0})
    count = 0
    for e in allEvents:
        count += 1
        if 'category' in e and 'description' in e and 'name' in e:
            allCategorizedEvents.append(e)
    modernEvents = reduceCategories(allCategorizedEvents)
    print count, "total events, learning from the", len(modernEvents), "categorized events"
    return modernEvents

## Event Categorization

In [144]:
def someCurrentCategories():
    """Looks at current events for the categories list, to be used if Facebook changes its events in the future"""
    allCategorizedEvents = []
    allEvents = events_collection.find({}, {"category": 1, "description": 1, "name": 1, "_id": 0})
    for e in allEvents:
        if 'category' in e and 'description' in e and 'name' in e:
            allCategorizedEvents.append(e)
    skTarget = [e['category'] for e in allCategorizedEvents]
    count = sorted(list(set(skTarget)))
    print(count)

In [ ]:
def reduceCategories(events):
    """OTHER will discarded from the training data"""
    categoryMapping = {
        u'BOOK': u'LITERATURE',
        u'COMEDY': u'COMEDY_PERFORMANCE',
        u'CLASS': u'OTHER',
        u'DINING': u'FOOD',
        u'FAMILY': u'OTHER',
        u'FESTIVAL': u'PARTY',
        u'FOOD_TASTING': u'FOOD',
        u'FUNDRAISER': u'CAUSE',
        u'LECTURE': u'OTHER',
        u'MOVIE': u'FILM',
        u'NEIGHBORHOOD': u'OTHER',
        u'NIGHTLIFE': u'OTHER',
        u'RELIGIOUS': u'RELIGION',
        u'VOLUNTEERING': u'CAUSE',
        u'WORKSHOP': u'OTHER'
    }
    
    for e in events:
        if e in categoryMapping:
            
    
    
    return events
    

In [145]:
curListOfCategories = [u'ART', u'CAUSE', u'COMEDY_PERFORMANCE', u'DANCE', u'DRINKS', u'FILM', u'FITNESS', u'FOOD',
                       u'GAMES', u'GARDENING', u'HEALTH', u'LITERATURE', u'MEETUP', u'MUSIC', u'NETWORKING', u'PARTY',
                       u'RELIGION', u'SHOPPING', u'SPORTS', u'THEATER', u'WELLNESS']
X = gatherCategorizedEvents()
skText = [e['name']+' '+e['description'] for e in X]
skTarget = [e['category'] for e in X]

5819 total events, learning from the 2907 categorized events


In [146]:
someCurrentCategories()

[u'ART', u'CAUSE', u'COMEDY_PERFORMANCE', u'CONFERENCE', u'DANCE', u'FILM', u'FITNESS', u'FOOD', u'GAMES', u'GARDENING', u'HEALTH', u'LITERATURE', u'MEETUP', u'MUSIC', u'NETWORKING', u'PARTY', u'SPORTS', u'THEATER', u'WELLNESS']


In [147]:
sorted(list(set(skTarget)))

[u'ART',
 u'BOOK',
 u'CAUSE',
 u'CLASS',
 u'COMEDY',
 u'COMEDY_PERFORMANCE',
 u'CONFERENCE',
 u'CRAFTS',
 u'DANCE',
 u'DINING',
 u'DRINK',
 u'FAMILY',
 u'FESTIVAL',
 u'FILM',
 u'FITNESS',
 u'FOOD',
 u'FOOD_TASTING',
 u'FUNDRAISER',
 u'GAMES',
 u'GARDENING',
 u'HEALTH',
 u'LECTURE',
 u'LITERATURE',
 u'MEETUP',
 u'MOVIE',
 u'MUSIC',
 u'NEIGHBORHOOD',
 u'NETWORKING',
 u'NIGHTLIFE',
 u'OTHER',
 u'PARTY',
 u'RELIGIOUS',
 u'SHOPPING',
 u'SPORTS',
 u'THEATER',
 u'VOLUNTEERING',
 u'WELLNESS',
 u'WORKSHOP']

In [116]:
from collections import Counter
for i in Counter(skTarget).keys():
    print(i)



NEIGHBORHOOD
MOVIE
FUNDRAISER
GAMES
NIGHTLIFE
FITNESS
LECTURE
FOOD_TASTING
FILM
CONFERENCE
SHOPPING
NETWORKING
VOLUNTEERING
DANCE
WELLNESS
DRINK
MEETUP
DINING
WORKSHOP
COMEDY
RELIGIOUS
CLASS
FOOD
LITERATURE
THEATER
FAMILY
COMEDY_PERFORMANCE
GARDENING
OTHER
MUSIC
PARTY
CAUSE
ART
FESTIVAL
SPORTS
BOOK
HEALTH
CRAFTS


## ML Helper Functions

In [77]:
from sklearn.cross_validation import train_test_split

def train(classifier, X, y, trails=25):
    scores = np.zeros(trails)
    for i in tqdm(range(0, trails)):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=i)

        classifier.fit(X_train, y_train)
        scores[i] = (classifier.score(X_test, y_test))
    print "Average Accuracy over %d trials: %s" % (trails, np.mean(scores))
    classifier.fit(X, y)
    return classifier
 

In [90]:
def predictSingle(vectorizer, classifier, x):
    print(x)
    x = vectorizer.transform(x)
    y_pred = classifier.predict(x)
    print(y_pred)


# Model Making

In [79]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
 
trial1 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB()),
])
 
nbModel = train(trial1, skText, skTarget)

100%|██████████| 25/25 [00:12<00:00,  1.97it/s]


Average Accuracy over 25 trials: 0.39807427785419536


In [80]:
trial2 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB(alpha=0.05)),
])

nbModel = train(trial2, skText, skTarget)

100%|██████████| 25/25 [00:12<00:00,  1.99it/s]


Average Accuracy over 25 trials: 0.5727647867950482


0.5709491059147179 without stoplist 

0.579697386519945 with my stoplist

In [62]:
stoplist = 'for a of the and to in ucla . / '.split()
stoplist = nltk.corpus.stopwords.words('english') + stoplist + string.punctuation.split()

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X = vectorizer.fit_transform(skText)

# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)

nbModel = MultinomialNB(alpha=0.05)
nbModel = train(nbModel, X, skTarget)

100%|██████████| 25/25 [00:00<00:00, 32.38it/s]

Average Accuracy over 25 trials: 0.582283356258597


0.5689133425034388 Tried PorterSteemer

0.5702338376891334 portersteemer with matchNotX

0.5700137551581843 just snowball stemmer

In [88]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize

def matchNotX(strg, search=re.compile(r'[^!#$%&()*+,-./:;<=>?@\\^_}|{~0123456789]').search):
    """make sure word has something than punctuation"""
    return bool(search(strg)) #make sure word has something other than punctuation

def stemming_tokenizer(text):
    stemmer = SnowballStemmer('english') #used to remove plural version of words
    tokenizer = word_tokenize
    return [stemmer.stem(w) for w in tokenizer(text) if matchNotX(w)]

vectorizer = TfidfVectorizer(tokenizer=stemming_tokenizer, stop_words='english') 
X = vectorizer.fit_transform(skText)
nbModel = MultinomialNB(alpha=0.05)
nbModel = train(nbModel, X, skTarget)

100%|██████████| 25/25 [00:00<00:00, 35.92it/s]

Average Accuracy over 25 trials: 0.5712792297111416


In [100]:
predictSingle(vectorizer, nbModel, skText[5:6])

[u'Travis Wall\'s Shaping Sound Under the Artistic Direction of Emmy Award Winner, TRAVIS WALL, and co-created with NICK LAZZARINI, TEDDY FORANCE and KYLE ROBINSON, SHAPING SOUND is an electrifying mash-up of dance styles and musical genres brought fully to life on stage by a dynamic company of contemporary dancers.\n\nThrough his Emmy Award winning work on "So You Think You Can Dance," Travis Wall has established himself as one of America\'s favorite choreographers. His artistic direction of SHAPING SOUND has already produced one spell-binding show in Dance Reimagined, which has captivated audiences all across North America. Now, Travis and SHAPING SOUND are back with their brand new show, After the Curtain. These visual musicians continue to dazzle audiences as they tell the story of a man fighting to find his creative voice after the death of his one true love. Heart-wrenching, breath-taking and ultimately uplifting, this is one dance show that audiences will remember forever.']
[u'

array([u'ART', u'BOOK', u'CAUSE', u'CLASS', u'COMEDY',
       u'COMEDY_PERFORMANCE', u'CONFERENCE', u'CRAFTS', u'DANCE',
       u'DINING', u'DRINK', u'FAMILY', u'FESTIVAL', u'FILM', u'FITNESS',
       u'FOOD', u'FOOD_TASTING', u'FUNDRAISER', u'GAMES', u'GARDENING',
       u'HEALTH', u'LECTURE', u'LITERATURE', u'MEETUP', u'MOVIE',
       u'MUSIC', u'NEIGHBORHOOD', u'NETWORKING', u'NIGHTLIFE', u'OTHER',
       u'PARTY', u'RELIGIOUS', u'SHOPPING', u'SPORTS', u'THEATER',
       u'VOLUNTEERING', u'WELLNESS', u'WORKSHOP'], dtype='<U18')

<2907x19447 sparse matrix of type '<type 'numpy.float64'>'
	with 167832 stored elements in Compressed Sparse Row format>

TESTING

In [ ]:
tokenizer = vectorizer.build_tokenizer()

for w in vectorizer.vocabulary_:
    if not matchNotX(w):
        print(w)